# The new kid on the block: DuckDB

Version 1.0 of DuckDB was released in 2024, it's pretty new and probably a must for data scientists.

In my opinion it's one of the most effective ways to process tabular data.

## What is it?

In short, is a tool to run SQL queries against files and dataframes.

The SQL dialect is extremely sophisticated and it can directly read and write pandas dataframes, CSV fils, parquet, JSON and others, locally of from URLs or S3.

You will have a complete lesson on SQL later on, and we'll meet again and see the pros and cons of thes files format.

For today, this is just an overview

In [1]:
import pandas as pd

# Create a DataFrame with sample data
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May'],
    'Sales': [100, 120, 140, 160, 180],
    'Expenses': [80, 90, 100, 110, 120],
    'Country': ['France', 'France', 'Samoa', 'Italy', 'Germany']
}
df = pd.DataFrame(data)
df

,Month,Sales,Expenses,Country
0,Jan,100,80,France
1,Feb,120,90,France
2,Mar,140,100,Samoa
3,Apr,160,110,Italy
4,May,180,120,Germany


In [6]:
import duckdb as ddb
ddb.query("""
SELECT *
FROM df
WHERE Month IN ('Jan', 'Feb')
""")

┌─────────┬───────┬──────────┬─────────┐
│  Month  │ Sales │ Expenses │ Country │
│ varchar │ int64 │  int64   │ varchar │
├─────────┼───────┼──────────┼─────────┤
│ Jan     │   100 │       80 │ France  │
│ Feb     │   120 │       90 │ France  │
└─────────┴───────┴──────────┴─────────┘

You can get a Pandas dataframe directly from DDB.

In [4]:

new_df = ddb.query("""
SELECT *, Sales - Expenses AS Profit
FROM df
WHERE Month IN ('Jan', 'Feb')
""").df()

new_df

,Month,Sales,Expenses,Country,Profit
0,Jan,100,80,France,20
1,Feb,120,90,France,30


DuckDB can easily read files from URLs and the local filesystem, and integrate data from them and from Dataframes.

See here how easily it can read a CSV and join it with the local dataframe, sorting the results as we please.

In this case since the CSV has no headers we can provide them.

In [5]:
countries_capitals_expenses = ddb.query("""
SELECT
    csvdata.*,
    df.*
FROM read_csv(
    "https://gist.githubusercontent.com/jacopofar/804c5694ac12a9d6fde653b5a6e3b983/raw/8ffd027bf5e9b1184695e1e55798f699e6acda74/countries_capitals.tsv",
    names = ['CountryName', 'CapitalName']
) csvdata
LEFT JOIN df
ON df.country == csvdata.CountryName
ORDER BY Expenses, CountryName
""").df()

countries_capitals_expenses

,CountryName,CapitalName,Month,Sales,Expenses,Country
0,France,Paris,Jan,100,80,France
1,France,Paris,Feb,120,90,France
2,Samoa,Apia,Mar,140,100,Samoa
3,Italy,Rome,Apr,160,110,Italy
4,Germany,Berlin,May,180,120,Germany
...,...,...,...,...,...,...
257,Yemen,Sana'a,None,<NA>,<NA>,None
258,Zambia,Lusaka,None,<NA>,<NA>,None
259,Zanzibar,Zanzibar,None,<NA>,<NA>,None
260,Zimbabwe,Harare,None,<NA>,<NA>,None


## Bonus exercise

There's a duplicated country in the CSV above. Can you find it?

You may use Pandas or DuckDB or both, there are many ways to solve it